# Lab 4: Logistic Regression, SVM, Cross Validation
In this lab we will 
1. get some more practice with classification methods(Logistic Regression and Support Vector Machines)
2. explore validation methods that we covered in class this week and last.

We'll use the credit data from Assignment 2 for this lab.

## Import libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
%matplotlib inline

In [30]:
# Load data
fname = 'data/credit-data.csv'
df = pd.read_csv(fname)

In [22]:
# get counts
df.count()

PersonID                                41016
SeriousDlqin2yrs                        41016
RevolvingUtilizationOfUnsecuredLines    41016
age                                     41016
zipcode                                 41016
NumberOfTime30-59DaysPastDueNotWorse    41016
DebtRatio                               41016
MonthlyIncome                           33042
NumberOfOpenCreditLinesAndLoans         41016
NumberOfTimes90DaysLate                 41016
NumberRealEstateLoansOrLines            41016
NumberOfTime60-89DaysPastDueNotWorse    41016
NumberOfDependents                      39979
dtype: int64

# 1) Cleaning the dataset
A decent chunk of the columns are missing values. If you're using features that are missing values, you need to do something to take care of them(IE: drop those rows, fill in the missing value, add a new category that corresponds to missing value, etc).

Remember that a lot of these features are not operating at the same magnitude. A feature like 
Monthly Income has values on the order of $10^3$ while a feature like Number of times 90 days late is on the order of 10. So do some kind of feature normalization or bucketing to account for this.

## Important
Some data cleaning can happen on the entire data set and some needs to happen for each training and test set separately. You can write the code/function to do this here but you should not do this globally on the entire dataset. For example:
* imputation: any imputation requiring computation (mean or median for example) on a given data set should take place for each training set separately.
* discretization: same as imputation.

In [14]:
# CODE

## Split the data 
Split the data into a training set and a test set. We're assuming here that there is no time dimension here so we'll split the data by rows and not use any time variables into account.

In [12]:
from sklearn.model_selection import train_test_split
features = [] # Pick the features you want
df_features = df[features]
df_target = df['SeriousDlqin2yrs']
X_train, X_test, Y_train, Y_test = train_test_split(df_features, df_target, test_size=0.2)

# 2) Applying Logistic Regression

Ref: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

You'll notice that LogisticRegression takes a ton of parameters. We'll play around with the "penalty" and "C" parameters.
If we set the penalty parameter to ['l2'](http://mathworld.wolfram.com/L2-Norm.html), sklearn's LogisticRegression model solves the following minimization problem:

$$ \min_{\beta} ||\beta||_2 + C \sum_{i} \log ( -y_i (X_i^T \beta) +1)$$

Similarly, if we set the penalty parameter to ['l1'](http://mathworld.wolfram.com/L2-Norm.html), LogisticRegression will solve the following minimization problem:

$$\min_{\beta} ||\beta||_1 + C \sum_{i} \log ( -y_i (X_i^T \beta) +1)$$

where $$||\beta||_2 = \sqrt { \sum_{i} \beta_i^2 }$$ and $$||\beta||_1 =  \sum_{i} | \beta_i | $$ 

Try running logistic regression with both L1 and L2 penalties and a mix of C values with K-fold cross validation on the training set. If the K-fold cross validation takes too long, consider using a smaller subset of the data. Write a function to perform the cross validation over both penalties, and a range of C values (something like $10^{-2}, 10^{-1}, 1, 10, 10^2)$ is reasonable

Write a function that performs [KFold cross validation](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) over the training set. This function
should loop over penalty('l1' and 'l2') and C values.

In this cross validation function, you should only be dealing with the training set. Below is a sample of how I might structure my cross validation + parameter search loop.

In [ ]:
# Here is an example of how I'd do this for linear regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error # to evaluate how good/bad my model predictions are
N = 1000
X = np.random.random((N, 10))
Y = np.random.random(N)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
alphas = [10**i for i in range(-3, 3)]
splits = 5
kf = KFold(n_splits=splits)

results = {} # dict of model params -> model performance over the KFold cross validation
for fold_num, (train_idx, test_idx) in enumerate(kf.split(X_train)):
    x_split_train, x_split_test = X_train[train_idx], X_train[test_idx]
    y_split_train, y_split_test = y_train[train_idx], y_train[test_idx]
    
    for a in alphas:
        linreg = Ridge(alpha=a)
        linreg.fit(x_split_train, y_split_train)
        y_pred = linreg.predict(x_split_test)
        model_key = (a, ) # this will be a longer tuple for things with more parameters
        results[a] =  results.get(a, 0) + mean_squared_error(y_pred, y_split_test) / splits

for model, model_perf in results.iteritems():
    # the MSE here is meaningless b/c we're fitting random noise to random noise.
    print("Model with params: {} | MSE: {:.2f}".format(model, model_perf))

For regression tasks, mean_squared_error is a sensible metric to evaluate on. 

For classification, we might not want to use accuracy if our dataset is a unbalanced. Instead we might use [roc_auc_score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score) or precision or recall at a certain threshold. Note that regardless of which you use, you should call predict_proba from the LogisticRegression object.

Write a function to do do cross validation over the parameters penalty and C with LogisticRegresion.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

c_values = [] # pick the C values

# YOUR CODE
def cv_logistic_regression(**params):
    kf = KFold(n_splits=5)
    pass

What combination of C value and penalty gives the highest cross validation score? Now create new models that uses the penalty and C-value that performed the best(according to whatever criteria you decided on). Fit this fresh model on the entire training set, and evaluate it on the test set.

To really see the difference between L1 and L2 regularization, we need to take a closer look at the models they produced. Plot a histogram of the weight values of LogisticRegression models for each C value. You can access these weight coefficients via the coef\_ attribute in LogisticRegression. Do you notice anything interesting happening as the C value varies?

You should see that the number of weight coefficients set to 0 in L1 regularized Logistic Regression increases with smaller C values. Revisit the minimization problems: you can think of the "C" parameter as a way of specifying the tradeoff between the L1/L2 penalty and the negative log likelihood of the model. A small value of "C" means that we are assigning greater weight to the L1/L2 penalty(equivalently downweighting the negative log likelihood).
**L1 regularization induces sparse models** - this can be a very useful if you suspect your classification target variable can be explained by few features.

Recall that a LogisticRegression model has a predict_proba function, so we can change our
classification threshold as we please. For tasks where a given class is extremely rare (IE: medical diagnoesis), you might have a very small number of positive examples(IE: patient has cancer). And you can artificially get a very high classification accuracy by simply predicting "No Cancer" for each patient. So instead of just using classification accuracy to see how good our model is, we'll consider precision and recall.

As a reminder, precision is defined as:
$$\text{precision} = \frac{TP}{TP + FP} $$
and recall is defined as:
$$ \text{recall} = \frac{TP}{TP + FN} $$

Complete the following functions to compute precision and recall.

In [5]:
def precision(y_pred, y_true):
    # YOUR CODE
    pass

def recall(y_pred, y_true):
    # YOUR CODE
    pass

LogisticRegression.predict uses 0.5 as the threshold for predicting True vs False. But a lower/higher probability threshold might give us better results. Try varying the classification probability thresholds and compute the precision and recall values and plot the [ROC curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html) for the LogisticRegression model that performed the best. Also compute the [AUC](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score).

# 3) Applying SVM

Ref: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
The SVM Classifier also takes quite a few parameters. We will be playing with following parameters:
* C: same as above
* kernel: string, it can be one of 'rbf', 'poly', 'linear'. More on that below:
* gamma: parameter for the rbf kernel
* degree: int, optional(only used if the kernel is 'poly'
* coef0: float, optional(only used if the kernel is ‘poly’ or ‘sigmoid’)
* probability: **set it to true** (this boolean flag enables/disables probability estimates)

SVM tries to find the hyperplane that maximizes the "margin" between the two classes of points. The "C" parameter in [SVC](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC) has the same role as the "C" parameter in LogisticRegression: it tells you how much to penalize the "size" of the weight vector. Note that SVC only allows for L2 regularization.

## Choosing the kernel
The kernels
* linear: $\langle x, x'\rangle$.
* polynomial: $(\gamma \langle x, x'\rangle + r)^d$. $d$ is specified by keyword degree, $r$ by coef0.
* rbf: $\exp(-\gamma \|x-x'\|^2)$.  $\gamma$ is specified by keyword gamma, must be greater than 0.
* sigmoid $(\tanh(\gamma \langle x,x'\rangle + r))$, where r is specified by coef0, and tanh is the [hyperbolic tangent](https://en.wikipedia.org/wiki/Hyperbolic_function#Definitions)

You should think of this kernel function as playing a similar role to what the distance metric  did in the KNN classifier. Fundamentally, SVM is trying to find a hyperplane that maximize some **geometric distance** between points of different classes, and this distance is determined by the kernel. So here it seems like there are many many free parameters to play with. To save a bit of time, the most important kernels to try out are: the linear, rbf kernel(with a few different gamma values). You should think about what a high/low gamma value does to the RBF kernel.

Write some code that computes the RBF kernel between two rows of the training set. Plot the rbf kernel between these two points with a few different values of gamma

In [ ]:
x1 = X_train[10] # any random training point
x2 = X_train[20] # any random training point that is not the same as x1
gammas = [10**i for i in np.arange(-1, 1, 0.05)]

def rbf(x1, x2, gamma):
    pass

rbf_vals = [rbf(x1, x2, g) for g in gammas]
plt.plot(gammas, rbf_vals)
plt.title('Gamma vs RBF')
plt.xlabel('Gamma')
plt.ylabel('RBF kernel value')
plt.show()

As we did for LogisticRegression, write a function/some code to perform KFold crossvalidation on the training set to find the optimal C and kernel parameters. Repeat the cross validation after normalizing your continous features as well. Does normalization affect the SVM performance?

In [9]:
from sklearn.svm import SVC
def cv_svm(**params):
    pass

The SVC classifier might take some time to train because it has to compute this square matrix of kernel values, which can be expensive for 30k points. If it's taking too long, you should subsample your data and try to fit the SVC model on this smaller subsample.

SVC also has a [predict_proba](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.predict_proba) function, so let's also plot an ROC curve for your best performing SVC model.

Do you think SVM or LogisticRegression is better suited for this classification task?

## 4) Class Imbalance and Evaluation Metrics

As noted in lecture, accuracy is not necessarily the best way to evaluate your classifier. This is especially true in situations where
our classification dataset has class imbalance. In our current dataset, we have the following class proportions:

Both LogisticRegression and SVC have a class_weight parameter that allows you to specify how much the model should prefer correctly classifying one class over another. Try specifying this parameter in your models and see how this affects the evaluation metrics you just tried above. 

## Additional Questions
If you have the time, here are some additional things to try:
* pull in other features from the ACS api like demographic information of a given zipcode. Does this improve the models performance? Do you think this is an ethical thing to do for deciding loan applications?
* you've written a cv_logistic_regression function and a cv_svm function which probably look very similar. Try writing a cross validation function that takes a model(IE: LogisiticRegression, SVC, DecisionTreeClassifier) as a parameter as well so that we can just do cv_model(SVC, ....) and cv_model(LogisticRegression, ...)